# Lex-Minimizer #
In this notebook, we will demonstrate the use of functions that computes the inf-minimizer (`CompInfMin`) and lex-minimizer (`CompLexMin`), as well as other experiments. You can find all source code in `lex.jl`.

In [1]:
pwd()

"/home/xshi0x63/Desktop/YINS/Laplacians.jl/notebooks"

In [3]:
push!(LOAD_PATH,"../src")
using Laplacians

In [4]:
using PyPlot

In [51]:
include("../src/lex.jl") # for development

ModDijkstra (generic function with 1 method)

### IterLex Functions ###
* `simIterLexUnwtd(...)` runs IterLex on a **uniformly weighted** graph for a specific number of iterations;

* `checkLexUnwtd(...)` checks the correctness of the assignment up to `LEX_EPS`;

* `simIterLex(...)` runs IterLex on any weighted graph;

* `checkLex(...)` checks the correctness of the assignment up to `LEX_EPS`;

## Simple examples ##
### Path Graphs ###
A path graph with `n` vertices. Let vertex `1` and `n` be terminals, with voltages `0` and `1` respectively.

In [52]:
# Set up
n = 10
Pn = pathGraph(n)

isTerm = zeros(Bool, n)
isTerm[1] = true
isTerm[n] = true

initVal = zeros(n)
initVal[n] = 1.0


1.0

In [53]:
# default error tolerance
LEX_EPS

1.0e-12

In [54]:
t1 = 5
asgnmt = simIterLexUnwtd(t1, Pn, isTerm, initVal)
@printf("After %d iterations: %s\n", t1, checkLexUnwtd(Pn, isTerm, initVal, asgnmt, fatal = false))

t2 = 500
asgnmt = simIterLexUnwtd(t2, Pn, isTerm, initVal)
@printf("After %d iterations: %s\n", t2, checkLexUnwtd(Pn, isTerm, initVal, asgnmt, fatal = false))


After 5 iterations: false
After 500 iterations: true


### Star Graph ###

In [55]:
# Star Graph
n = 5
Sn = zeros(n, n)
Sn[1,:] = a = [0, 1/20, 1/20, 1/10, 1/18]
Sn[:,1] = a'
Sn = sparse(Sn)

isTerm = ones(Bool, n)
isTerm[1] = false

initVal = [0.0, 20, -5, -5, 17]

asgnmt = simIterLex(1, Sn, isTerm, initVal)

5-element Array{Float64,1}:
  3.33333
 20.0    
 -5.0    
 -5.0    
 17.0    

In [56]:
checkLex(Sn, isTerm, initVal, asgnmt, fatal = false)

true